# Traveling Salesperson Problem

The Traveling Salesperson Problem (TSP) is a classic optimization problem in computer science and operations research. It involves finding the most efficient route for a salesperson to visit a set of cities and return to the starting city, with the goal of minimizing travel distance or cost.

## Problem Definition
Given:
- A list of **cities** (or nodes).
- The **distances or costs** between each pair of cities.

Objective:
- Find the shortest possible route that visits each city **exactly once** and returns to the starting city.

#### Example
- Suppose there are four cities: A, B, C, and D.
- The distances between them are:
  - A → B: 10
  - A → C: 15
  - A → D: 20
  - B → C: 35
  - B → D: 25
  - C → D: 30
  
| From/To | A   | B   | C   | D   |
|---------|-----|-----|-----|-----|
| A       | n/a | 10  | 15  | 20  |
| B       | 10  | n/a | 35  | 25  |
| C       | 15  | 35  | n/a | 30  |
| D       | 20  | 25  | 30  | n/a |


The task is to determine the sequence of cities (e.g., A → B → C → D → A) that minimizes the total distance traveled.

## Applications
1. **Logistics and Delivery**: Optimizing routes for delivery trucks or couriers.
2. **Manufacturing**: Designing efficient robotic arms or conveyor belts.
3. **Tourism**: Creating optimal travel itineraries.

## Complexity
- **Computationally Hard**: TSP is NP-hard, meaning there's no known polynomial-time algorithm to solve it for all cases.
- For \(n\) cities, there are \((n-1)!\) possible routes.

## Solution Approaches
1. **Exact Algorithms**:
   - **Brute Force**: Explore all possible routes (impractical for large \(n\)).
   - **Dynamic Programming**.
   - **Mixed Integer Linear Programming (MILP)**.

2. **Heuristic and Approximation Methods**:
   - **Greedy Algorithms**.
   - **Genetic Algorithms**.
   - **Simulated Annealing**.
   - **Ant Colony Optimization**.

3. **Machine Learning**

## Mixed-Integer Linear Program (MILP) Formulation

The **Traveling Salesperson Problem (TSP)** can be formulated as a **Mixed-Integer Linear Program (MILP)**. Below is a typical formulation:

### Sets
- $i$, $j$: Cities (nodes).

### Parameters
- $ c_{ij} $: Cost or distance of traveling from city $i$ to city $j$.
- $ n $: Number of cities.

### Variables
1. **Decision Variables**:
   - $ x_{ij} \in \{0, 1\} $: Binary variable indicating whether the path from city $i$ to city $j$ is included in the tour.
     - $ x_{ij} = 1 $: Path from city $i$ to $j$ is used.
     - $ x_{ij} = 0 $: Path from city $i$ to $j$ is not used.

2. **Auxiliary Variables** (for subtour elimination):
   - $ u_i $: A continuous variable representing the order in which city $i$ is visited, used to eliminate subtours.

### Objective Function:
Minimize the total travel cost:
$$
\min_{x_{ij}, u_i} \quad \sum_{i=1}^{n} \sum_{j=1, j \neq i}^{n} c_{ij} x_{ij}
$$

### Constraints:
1. **Each city is entered exactly once**:
   $$
   \sum_{i=1, i \neq j}^{n} x_{ij} = 1 \quad \forall j = 1, \dots, n
   $$

2. **Each city is exited exactly once**:
   $$
   \sum_{j=1, j \neq i}^{n} x_{ij} = 1 \quad \forall i = 1, \dots, n
   $$

3. **Subtour elimination constraints** (Miller-Tucker-Zemlin formulation):
   $$
   u_i - u_j + n \cdot x_{ij} \leq n - 1 \quad \forall i, j = 2, \dots, n, \; i \neq j
   $$
   $$
   1 \leq u_i \leq n - 1 \quad \forall i = 2, \dots, n
   $$

   Here, $u_i$ ensures that a valid sequence of cities is followed, preventing subtours. The basic formulation with decision variables $x_{ij}$ ensures each city is visited exactly once but does not inherently prevent **subtours** (i.e., tours that visit only a subset of cities). These constraints force the auxiliary variable $u_i$ to ensure a valid tour. If $x_{ij} = 1$ (i.e., there is a path from $i$ to $j$), then $u_i < u_j$.


